In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import common as common
import pandas as pd

In [17]:
class DeepLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(DeepLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True, dropout=0.5)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        out = self.fc(x)
        return out


In [18]:
train_df = common.get_train_df()
train_df = common.over_sample_df(train_df)
target_column, feature_column = common.get_column_names(train_df)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_df[target_column])

scaler = StandardScaler()
X = scaler.fit_transform(train_df[feature_column])

X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
y = torch.tensor(y, dtype=torch.long)

common.print_class_distribution(y)

3    0.142857
2    0.142857
5    0.142857
4    0.142857
0    0.142857
1    0.142857
6    0.142857
Name: proportion, dtype: float64


In [19]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)


# Model parameters
input_size = X.shape[2]
hidden_size = 512
num_layers = 3
num_classes = len(label_encoder.classes_)
model = DeepLSTMModel(input_size, hidden_size, num_layers, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)


# Training loop with early stopping
epochs = 50
best_acc = 0
# patience, counter = 10, 0
for epoch in range(epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    
    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Validation Accuracy: {accuracy:.2f}% {loss}")
    scheduler.step(accuracy)
    
    # # Early Stopping
    # if accuracy > best_acc:
    #     best_acc = accuracy
    #     counter = 0
    # else:
    #     counter += 1
    #     if counter >= patience:
    #         print("Early stopping triggered.")
    #         break


/opt/anaconda3/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/50], Validation Accuracy: 15.14% 1.9418785572052002
Epoch [2/50], Validation Accuracy: 16.29% 1.9342423677444458
Epoch [3/50], Validation Accuracy: 16.57% 1.9231781959533691
Epoch [4/50], Validation Accuracy: 19.14% 1.8937811851501465
Epoch [5/50], Validation Accuracy: 21.71% 1.876651406288147
Epoch [6/50], Validation Accuracy: 24.86% 1.824602484703064
Epoch [7/50], Validation Accuracy: 28.29% 1.684921383857727
Epoch [8/50], Validation Accuracy: 29.43% 1.5216585397720337
Epoch [9/50], Validation Accuracy: 36.86% 1.5215129852294922
Epoch [10/50], Validation Accuracy: 40.00% 1.4107234477996826
Epoch [11/50], Validation Accuracy: 44.57% 1.363008737564087
Epoch [12/50], Validation Accuracy: 46.57% 1.1611924171447754
Epoch [13/50], Validation Accuracy: 52.29% 1.1093069314956665
Epoch [14/50], Validation Accuracy: 55.71% 0.8943093419075012
Epoch [15/50], Validation Accuracy: 59.14% 0.8272336721420288
Epoch [16/50], Validation Accuracy: 61.43% 0.7049026489257812
Epoch [17/50], Valida

In [20]:
test_df = pd.read_csv("data.csv")
X_test = scaler.transform(test_df[feature_column])
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)

model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_predictions = torch.argmax(test_outputs, axis=1)
    test_predictions = label_encoder.inverse_transform(test_predictions.numpy())

output_df = pd.DataFrame({"ID": test_df["ID"], "y_pred": test_predictions, "y_true": test_df[target_column]})


In [21]:
common.get_accuracy(output_df)

Accuracy: 13.98%
